<a href="https://colab.research.google.com/github/javihm77/Data-Science/blob/master/Zubale_Test_Javier_Hernandez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge 1

We have two CSV files: one for products and the other for orders, with the following structure.
products.csv:
-id
-name
-category
-price

orders.csv:
-id
-product_id
-quantity
-created_date
Please note that every order can only contain one product type.
We need to combine both files to consolidate the information in a single CSV file in the following columns:
-order_created_date
-order_id
-product_name
-quantity
-total_price
This data has to be stored in a newly created file named “order_full_information.csv”.

In [1]:
import pandas as pd
import numpy as np

orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

orders.head()

,id,product_id,quantity,created_date
0,1,11,1,'2024-12-01'
1,2,17,2,'2024-12-01'
2,3,19,1,'2024-12-01'
3,4,12,1,'2024-12-01'
4,5,11,2,'2024-12-01'


In [2]:
products.head(6)

,id,name,category,price
0,1,Product_1,Pants,92.55
1,2,Product_2,Shirts,43.11
2,3,Product_3,Jackets,59.02
3,4,Product_4,Shoes,49.65
4,5,Product_5,Pants,44.59
5,6,Product_6,Shoes,16.27


In [3]:
orders.info()
print(" ")
print(" ")
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            50 non-null     int64 
 1   product_id    50 non-null     int64 
 2   quantity      50 non-null     int64 
 3   created_date  50 non-null     object
dtypes: int64(3), object(1)
memory usage: 1.7+ KB
 
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        20 non-null     int64  
 1   name      20 non-null     object 
 2   category  20 non-null     object 
 3   price     20 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 772.0+ bytes


In [4]:
merged_df = pd.merge(orders, products, left_on='product_id', right_on='id', how='inner')
merged_df['total_price'] = merged_df['quantity'] * merged_df['price']

new_column_names = {
    'created_date': 'order_created_date',
    'id_x': 'order_id',
    'name': 'product_name'
}
desired_order = ['order_created_date', 'order_id', 'product_name', 'quantity', 'total_price']

sales = merged_df.rename(columns=new_column_names)
sales = sales.reindex(columns=desired_order)
sales.sort_values(by='order_id', ascending=True, inplace=True)

In [5]:
sales.to_csv('order_full_information.csv', index=False)
sales.head()

,order_created_date,order_id,product_name,quantity,total_price
0,'2024-12-01',1,Product_11,1,69.06
1,'2024-12-01',2,Product_17,2,197.02
2,'2024-12-01',3,Product_19,1,51.94
3,'2024-12-01',4,Product_12,1,50.99
4,'2024-12-01',5,Product_11,2,138.12


# Challenge 2

Due to an error in one of our systems, the price information from orders.csv came in Brazilian currency (BRL), and we need to convert that value to US dollars.
Get the latest currency data from: https://app.freecurrencyapi.com/
You will need to get an API key from the website.

Include a step in your code to get this information, and use the data needed to get the total price in the desired currency. Persist the results in a new file “fixed_order_full_information.csv”, with the following columns:
-order_created_date
-order_id
-product_name
-quantity
-total_price_br
-total_price_us

In [6]:
!pip install freecurrencyapi

In [7]:
import freecurrencyapi
from google.colab import userdata

api_key = userdata.get('FreeCurrencyAPI')
client = freecurrencyapi.Client(api_key)

print(client.status())

result = client.latest(base_currency=['BRL'],currencies=['USD'])
conversion = result['data']['USD']
print("Conversion rate BRL to USD",conversion)


{'account_id': 449773735914901504, 'quotas': {'month': {'total': 5000, 'used': 14, 'remaining': 4986}, 'grace': {'total': 0, 'used': 0, 'remaining': 0}}}
Conversion rate BRL to USD 0.1764327847


In [8]:
new_column_names = {
    'total_price': 'total_price_br'
}

sales = sales.rename(columns=new_column_names)
sales['total_price_us'] = round(sales['total_price_br'] * conversion,2)
sales.head()
sales.to_csv('fixed_order_full_information.csv', index=False)

Now we want to explore a little with our data. Use Python to find the following information:

1.   Date where we create the max amount of orders.
2.   Most demanded product and the total sell price.
3.   The top 3 most demanded categories.

Store the results in a single CSV file named: kpi_product_orders.csv

In [9]:
max_day_orders = sales.groupby('order_created_date')['order_id'].count()
max_day_orders = max_day_orders.idxmax()
max_day_orders

"'2024-12-06'"

In [10]:
most_demanded_product = sales.groupby('product_name')['quantity'].sum()
most_demanded_product = most_demanded_product.idxmax()

total_sell_price = sales[sales['product_name'] == most_demanded_product]['total_price_br'].sum()

print("Most demanded product: ",most_demanded_product,"Total sell price: ",round(total_sell_price,2))

Most demanded product:  Product_5 Total sell price:  891.8


In [11]:
merged_df_category = merged_df.groupby('category')['quantity'].sum()
merged_df_category = merged_df_category.nlargest(3)
merged_df_category

,quantity
category,
Shirts,50
Jackets,30
Pants,29


In [12]:
kpi = pd.DataFrame({'Day with maximum orders': [max_day_orders],
                    'most_demanded_product': [most_demanded_product],
                    'total_sell_price': [round(total_sell_price,2)],
                    'top_3_most_demanded_categories': [merged_df_category]})
kpi.to_csv('kpi_product_orders.csv', index=False)
kpi

,Day with maximum orders,most_demanded_product,total_sell_price,top_3_most_demanded_categories
0,'2024-12-06',Product_5,891.8,category Shirts 50 Jackets 30 Pants ...


# Challenge 3

Instead of using the CSV files, create a PostgreSQL database and create the tables using these files as data sources.
Programmatically do this step (using a script).


In [13]:
!pip install psycopg2-binary

In [14]:
import psycopg2

db_host = 'testdb.ckri64804wdp.us-east-1.rds.amazonaws.com'
db_user = 'javihm77'
db_password = 'nPGKMFLFdRTk6DVcTdQX'
db_name_to_connect = 'postgres'
db_port = '5432'
db_name_to_create = 'sales'

conn = None
cursor = None

try:
    # Connect to the PostgreSQL server
    conn = psycopg2.connect(host=db_host, database=db_name_to_connect, user=db_user, password=db_password,port=db_port)
    conn.autocommit = True
    cursor = conn.cursor()

    # Check if the database exists
    cursor.execute(f"SELECT 1 FROM pg_database WHERE datname = '{db_name_to_create}';")
    exists = cursor.fetchone()

    if not exists:
        # Create the database if it doesn't exist
        sql_create_database = f"CREATE DATABASE {db_name_to_create};"
        cursor.execute(sql_create_database)

        print(f"Database '{db_name_to_create}' created successfully.")
    else:
        print(f"Database '{db_name_to_create}' already exists.")

    sql_drop_tables = """
    DROP TABLE IF EXISTS orders;
    DROP TABLE IF EXISTS products;
    """
    cursor.execute(sql_drop_tables)

    sql_create_table_products = """
    CREATE TABLE IF NOT EXISTS products (
        id INT PRIMARY KEY,
        name VARCHAR(255),
        category VARCHAR(255),
        price DECIMAL(10, 5)
    );
    """

    sql_create_table_orders = """
    CREATE TABLE IF NOT EXISTS orders (
        id INT PRIMARY KEY,
        product_id INT,
        quantity INT,
        created_date DATE,
        FOREIGN KEY (product_id) REFERENCES products(id)
    );
    """

    cursor.execute(sql_create_table_products)
    cursor.execute(sql_create_table_orders)

    print("Tables 'products' and 'orders' created successfully.")

    # Now, insert the data
    insert_into_products = """
    INSERT INTO products (id, name, category, price)
    VALUES (%s, %s, %s, %s);
    """

    for index, row in products.iterrows():
        values = (row['id'], row['name'], row['category'], row['price'])
        cursor.execute(insert_into_products, values)

    insert_into_orders = """
    INSERT INTO orders (id, product_id, quantity, created_date)
    VALUES (%s, %s, %s, %s);
    """

    for index, row in orders.iterrows():
        values = (row['id'], row['product_id'], row['quantity'], row['created_date'])
        cursor.execute(insert_into_orders, values)

    print("Data inserted successfully.")

except Exception as e:
    print(f"Error connecting database: {e}")

#finally:
    # Close the connection
#    if cursor:
#        cursor.close()
#    if conn:
#        conn.close()

Database 'sales' already exists.
Tables 'products' and 'orders' created successfully.
Data inserted successfully.


Use SQL to get the information for each of the previous points:

1.   The date with max amount of orders
2.   The most demanded product
3.   The top 3 most demanded categories

(Note: query efficiency will be taken into account, regardless of the amount of data these tables have, imagine that the productive environment has millions of records)

In [15]:
import psycopg2

conn = None
cursor = None

try:
  conn = psycopg2.connect(host=db_host, database=db_name_to_connect, user=db_user, password=db_password, port=db_port)
  cursor = conn.cursor()

  sql_max_date = "SELECT created_date, count(*) FROM orders group by 1 order by 2 desc limit 1;"
  cursor.execute(sql_max_date)

  # Fetch all the results
  results = cursor.fetchall()

  # Get the column names from the cursor description
  column_names = [desc[0] for desc in cursor.description]

  # Create a pandas DataFrame from the results and column names
  products_df = pd.DataFrame(results, columns=column_names)

  # Display the DataFrame (or process it as needed)
  print("- The date with max amount of orders:")
  print(products_df)

  sql_demanded_products = """
  SELECT name as product
    , sum(quantity) as total_quantity
    , sum(quantity * price) as total_price
  FROM orders
    inner join products on orders.product_id = products.id
  group by 1
  order by 2 desc
  limit 3
  ;
  """

  cursor.execute(sql_demanded_products)
  results = cursor.fetchall()
  column_names = [desc[0] for desc in cursor.description]
  products_df = pd.DataFrame(results, columns=column_names)
  print("\n \n - The most demanded product:")
  print(products_df)

  sql_top_3_categories = """
  SELECT category
    , sum(quantity) as total_quantity
  FROM orders
    inner join products on orders.product_id = products.id
  group by 1
  order by 2 desc
  limit 3
  ;
  """
  cursor.execute(sql_top_3_categories)
  results = cursor.fetchall()
  column_names = [desc[0] for desc in cursor.description]
  products_df = pd.DataFrame(results, columns=column_names)
  print("\n \n - The top 3 most demanded categories:")
  print(products_df)


except Exception as e:
  print(f"Error checking tables: {e}")

finally:
  if cursor:
    cursor.close()
  if conn:
    conn.close()

- The date with max amount of orders:
  created_date  count
0   2024-12-06     10

 
 - The most demanded product:
      product  total_quantity total_price
0   Product_5              20   891.80000
1  Product_19              19   986.86000
2  Product_13              13   201.89000

 
 - The top 3 most demanded categories:
  category  total_quantity
0   Shirts              50
1  Jackets              30
2    Pants              29


# Challenge 4



1.   What other insights do you think would add value to the business that can be extracted using at least one of these tables?
Pick up to three, and explain why they might be useful and how we can get them. (If more tables are needed, list them.)

R/ 1-Sales trends by category by time aggregation: to identify seasonal trends, this affects inventory and the marketing strategies we can do in advance.
2-Average order value: key metric to identify is customers are purchasing more, can indicate the effectivines of upselling or cross selling.
3- PROFIT: I would add the cost of the product and obtain the difference between revenue minus cost, and do an analysis based on profit by category and product.

2.   What ETL/ELT tool would you use to extract this data and insert it into BigQuery? Explain the steps of creating this type of pipeline.

R/ I would use dataflow in GCP for data processing,  and stored procedures for transformation, I would also consider Apache Airflow.

3.   What AI-based pipeline could you add to this pipeline? Describe it.

R/ Sales forecasting pipeline, using the historical data we already have in BigQuery to predict future sales trends. Another idea would be a recommendation pipeline to recommend products to customers, adding it in the interface where they purchase.